# Reconstruction Sandbox
This notebook is a test-bed for regularization and reconstruction methods

In [18]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

# Load motiondeblur module and Dataset class
import libwallerlab.projects.motiondeblur as md
from libwallerlab.utilities.io import Dataset, isDataset
from libwallerlab.utilities import registration

# Platform imports
import os, glob
from os.path import expanduser

# Debugging imports
import llops as yp
import matplotlib.pyplot as plt
import numpy as np
import llops.operators as ops

yp.config.setDefaultBackend('numpy')
yp.config.setDefaultDatatype('float32')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Data

In [65]:
# # Beads Data
# dataset_path = './'
# header = 'beads'
# file_list_sns = sorted(glob.glob(os.path.join(dataset_path, '*beads2_raster_45ms_stopandstare_stop_and_stare_2019_02_04_16_43_09strip*.npz')))
# len(file_list_sns)

# Define user for path setting
dataset_path = './'
header = 'einstein'
file_list_sns = sorted(glob.glob(os.path.join(output_path, '*SNS_strip*.npz')))
len(file_list_sns)

35

In [66]:
file_list_sns

['/Users/zfphil/datasets/motiondeblur/output/163c_SNS_strip=0_regularize=[tv5.0e-04].npz',
 '/Users/zfphil/datasets/motiondeblur/output/163c_SNS_strip=10_regularize=[tv5.0e-04].npz',
 '/Users/zfphil/datasets/motiondeblur/output/163c_SNS_strip=11_regularize=[tv5.0e-04].npz',
 '/Users/zfphil/datasets/motiondeblur/output/163c_SNS_strip=12_regularize=[tv5.0e-04].npz',
 '/Users/zfphil/datasets/motiondeblur/output/163c_SNS_strip=13_regularize=[tv5.0e-04].npz',
 '/Users/zfphil/datasets/motiondeblur/output/163c_SNS_strip=14_regularize=[tv5.0e-04].npz',
 '/Users/zfphil/datasets/motiondeblur/output/163c_SNS_strip=15_regularize=[tv5.0e-04].npz',
 '/Users/zfphil/datasets/motiondeblur/output/163c_SNS_strip=16_regularize=[tv5.0e-04].npz',
 '/Users/zfphil/datasets/motiondeblur/output/163c_SNS_strip=17_regularize=[tv5.0e-04].npz',
 '/Users/zfphil/datasets/motiondeblur/output/163c_SNS_strip=18_regularize=[tv5.0e-04].npz',
 '/Users/zfphil/datasets/motiondeblur/output/163c_SNS_strip=19_regularize=[tv5.0e

## Load Strips

In [60]:
data_list = []
for strip_index in yp.display.progressBar(range(len(file_list_sns)), name='Segments Leaded'):
    object_sns = np.load(file_list_sns[strip_index])['object_recovered']
    data_list.append(object_sns)

## Loop over strips and generate data

In [61]:
# Define desired measurement shape
measurement_shape = (128, 2048)

In [62]:
x_list = []
for strip_index in yp.display.progressBar(range(len(data_list))):

    # Extract strips
    x = data_list[strip_index]

    # Normalize
    x /= yp.scalar(yp.mean(x))

    # Compute indicies to section array into
    n = [y_sh // m_sh for (m_sh, y_sh) in zip(measurement_shape, yp.shape(x))]
    x_indicies = np.arange(measurement_shape[1], yp.shape(x)[1], measurement_shape[1])
    y_indicies = np.arange(measurement_shape[0], yp.shape(x)[0], measurement_shape[0])

    # Split static (input) array in 2D blocks in x
    _x = np.array_split(x, x_indicies, axis=1)[:-1]

    # Split static (input) array in 2D blocks in y
    for __x in _x:
        x_list += np.array_split(__x, y_indicies, axis=0)[:-1]

In [63]:
compress_output = False

output_folder = '/Users/zfphil/datasets/motiondeblur/learning_data_sns'

if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    
# Save data
for index in yp.display.progressBar(range(len(x_list))):
    sns = x_list[index]
    
    # Determine output filename
    filename = '%s/%s_subframe=%d.npz' % (output_folder, header, index)
    
    # Save data
    if compress_output:
        np.savez_compressed(filename, measurement=sns)
    else:
        np.savez(filename, measurement=sns)
        
print('Saved %d files to %s' % (len(x_list), output_folder))

Saved 4760 files to /Users/zfphil/datasets/motiondeblur/learning_data_sns
